In [ ]:
from robovast.common.analysis import read_output_files, read_output_csv

# Configure paths
DATA_DIR = ''

# Read all CSV files into a combined dataframe
combined_df = read_output_files(DATA_DIR, lambda test_dir: read_output_csv(test_dir, "out.csv", skiprows=1))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Optional, Sequence, Tuple

# -----------------------
# Helpers
# -----------------------
def _validate_df(df: pd.DataFrame):
    expected = {'time', 'population', 'test', 'config', 'growth_rate', 'initial_population'}
    missing = expected - set(df.columns)
    if missing:
        raise ValueError(f"DataFrame missing expected columns: {missing}")
    df = df.copy()
    df['time'] = df['time'].astype(int)
    df['population'] = df['population'].astype(float)
    for cat in ['test', 'config', 'growth_rate', 'initial_population']:
        df[cat] = df[cat].astype('category')
    return df

def _pivot_config_to_runs(df: pd.DataFrame, config: str) -> pd.DataFrame:
    sel = df[df['config'] == config]
    if sel.empty:
        return pd.DataFrame()
    pivot = sel.pivot(index='time', columns='test', values='population').sort_index()
    return pivot

def _agg_config_stats(df: pd.DataFrame):
    configs = sorted(df['config'].cat.categories)
    rows = []
    for v in configs:
        pivot = _pivot_config_to_runs(df, v)
        if pivot.empty:
            rows.append({'config': v, 'n_runs':0, 'final_mean':np.nan, 'final_std':np.nan, 'max_mean':np.nan})
            continue
        final_t = pivot.index.max()
        finals = pivot.loc[final_t].dropna().values
        rows.append({
            'config': v,
            'n_runs': pivot.shape[1],
            'final_mean': np.nanmean(finals),
            'final_std': np.nanstd(finals, ddof=1),
            'max_mean': np.nanmean(pivot.max(axis=0))
        })
    return pd.DataFrame(rows).set_index('config')

def compute_ecdf(values):
    """Return x and y for an ECDF plot."""
    values = np.sort(values)
    n = len(values)
    y = np.arange(1, n+1) / n
    return values, y

# -----------------------
# Plotting functions
# -----------------------
def plot_mean_ribbon_configs(df, configs=None, q_lo=0.1, q_hi=0.9, figsize=(10,5), ax=None):
    df = _validate_df(df)
    all_configs = sorted(df['config'].cat.categories)
    if configs is None:
        configs = all_configs
    configs = [v for v in configs if v in all_configs]
    if ax is None:
        fig, ax = plt.subplots(figsize=figsize)
    cmap = plt.get_cmap('tab10')
    for i, v in enumerate(configs):
        pivot = _pivot_config_to_runs(df, v)
        if pivot.empty: continue
        mean_traj = pivot.mean(axis=1)
        lo = pivot.quantile(q_lo, axis=1)
        hi = pivot.quantile(q_hi, axis=1)
        color = cmap(i % 10)
        ax.plot(mean_traj.index, mean_traj.values, label=f"{v} (n={pivot.shape[1]})", color=color, linewidth=2)
        ax.fill_between(mean_traj.index, lo, hi, alpha=0.2, color=color)
    ax.set_xlabel('time')
    ax.set_ylabel('population')
    ax.set_title(f"Mean trajectories ± {int(100*q_lo)}-{int(100*q_hi)} percentile")
    ax.legend(loc='best', fontsize='small')
    ax.grid(alpha=0.2)
    return ax

def plot_final_boxplots_by_config(df, configs=None, figsize=(8,6), ax=None):
    df = _validate_df(df)
    all_configs = sorted(df['config'].cat.categories)
    if configs is None:
        configs = all_configs
    finals_by_config = []
    labels = []
    for v in configs:
        pivot = _pivot_config_to_runs(df, v)
        if pivot.empty: continue
        final_t = pivot.index.max()
        finals = pivot.loc[final_t].dropna().values
        finals_by_config.append(finals)
        labels.append(v)
    if ax is None:
        fig, ax = plt.subplots(figsize=figsize)
    ax.boxplot(finals_by_config, tick_labels=labels, showfliers=False)
    ax.set_xlabel('config'); ax.set_ylabel('final population')
    ax.set_title('Final population distribution by config')
    ax.grid(alpha=0.2)
    plt.xticks(rotation=45, ha='right')
    return ax

def plot_ecdf_final_by_config(df, configs=None, figsize=(8,5), ax=None):
    df = _validate_df(df)
    all_configs = sorted(df['config'].cat.categories)
    if configs is None:
        configs = all_configs
    if ax is None:
        fig, ax = plt.subplots(figsize=figsize)
    cmap = plt.get_cmap('tab10')
    for i, v in enumerate(configs):
        pivot = _pivot_config_to_runs(df, v)
        if pivot.empty: continue
        finals = pivot.loc[pivot.index.max()].dropna().values
        if finals.size == 0: continue
        x, y = compute_ecdf(finals)
        ax.step(x, y, where='post', label=f"{v} (n={len(finals)})", color=cmap(i%10))
    ax.set_xlabel('final population')
    ax.set_ylabel('ECDF')
    ax.set_title('ECDF of final populations by config')
    ax.grid(alpha=0.2)
    ax.legend(fontsize='small')
    return ax

def plot_config_category_scatter(df, configs=None, ax=None):
    """Scatter plot of growth_rate (x) vs initial_population (y) for each config, labeled by config name."""
    df = _validate_df(df)
    all_configs = sorted(df['config'].cat.categories)
    if configs is None:
        configs = all_configs
    if ax is None:
        fig, ax = plt.subplots(figsize=(8,6))
    # Get one row per config
    config_rows = [df[df['config'] == v].iloc[0] for v in configs]
    x = [float(row['growth_rate']) for row in config_rows]
    y = [float(row['initial_population']) for row in config_rows]
    names = configs
    ax.scatter(x, y, s=80, c='tab:blue', alpha=0.7)
    for xi, yi, name in zip(x, y, names):
        ax.text(xi, yi, str(name), fontsize=9, ha='left', va='bottom', fontweight='bold', color='tab:gray')
    ax.set_xlabel('growth_rate')
    ax.set_ylabel('initial_population')
    ax.set_title('Config Parameter Combinations')
    ax.grid(alpha=0.2)
    return ax

# -----------------------
# Main dashboard
# -----------------------
def compare_configs_dashboard(df, configs=None, max_small_multiple=6, q_lo=0.1, q_hi=0.9, heatmap_bins=60, 
                              two_column_text=False, save_path:Optional[str]=None):
    df = _validate_df(df)
    all_configs = sorted(df['config'].cat.categories)
    if configs is None:
        configs = all_configs
    configs = [v for v in configs if v in all_configs]
    
    summary_df = _agg_config_stats(df).reindex(configs)
    
    fig = plt.figure(constrained_layout=True, figsize=(16,12))
    gs = fig.add_gridspec(2,2)
    ax_mean = fig.add_subplot(gs[0,0])
    ax_box  = fig.add_subplot(gs[0,1])
    ax_ecdf = fig.add_subplot(gs[1,0])
    ax_scatter = fig.add_subplot(gs[1,1])
    
    plot_mean_ribbon_configs(df, configs=configs, q_lo=q_lo, q_hi=q_hi, ax=ax_mean)
    plot_final_boxplots_by_config(df, configs=configs, ax=ax_box)
    plot_ecdf_final_by_config(df, configs=configs, ax=ax_ecdf)
    plot_config_category_scatter(df, configs=configs, ax=ax_scatter)
    
    fig.suptitle("Multi-config Population Dashboard", fontsize=16)
    
    if save_path:
        fig.savefig(save_path, dpi=200, bbox_inches='tight')
        print(f"Dashboard saved to {save_path}")
    
    plt.show()
    
    return summary_df


summary_df = compare_configs_dashboard(combined_df,
                                        max_small_multiple=6,  # how many heatmaps to show
                                        q_lo=0.10,             # lower percentile for ribbons
                                        q_hi=0.90,             # upper percentile
                                        heatmap_bins=60,       # bins for density heatmaps
                                        two_column_text=True, # use two-column layout for configs
                                        save_path=None)        # or a filename to save images
